In [35]:
from nba_api.stats.endpoints import leaguedashlineups
from nba_api.stats.endpoints import leaguedashplayerstats
import statsmodels.api as sm
import pandas as pd
import numpy as np
import time

In [37]:
def LineupData(season,per):
    tempdf1 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Base', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    tempdf2 = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Advanced', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    tempdf = pd.merge(tempdf1,tempdf2[['GROUP_ID','PIE']].copy(), on='GROUP_ID', how='outer')
    return(tempdf)

In [38]:
def PlayerData(season,per):
    tempdf1 = leaguedashplayerstats.LeagueDashPlayerStats(measure_type_detailed_defense = 'Base', 
                                                 season = season, 
                                                 per_mode_detailed = per).get_data_frames()[0]
    return(tempdf1)

In [39]:
dfLineup = LineupData('2017-18','Per100Possessions')
dfPlayer = PlayerData('2017-18','Per100Possessions')
dfLineup.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,PIE
0,Lineups,-201144-201188-203516-1626185-1628415-,M. Conley - M. Gasol - J. Ennis III - J. Marti...,1610612763,MEM,11,6,5,0.545,53.6,...,5620,13689,4609,2033,15111,14573,3377,4449,5371,0.529
1,Lineups,-201144-201188-203516-1626150-1626185-,M. Conley - M. Gasol - J. Ennis III - A. Harri...,1610612763,MEM,8,5,3,0.625,52.9,...,5023,12721,5067,2557,15571,14018,3214,4841,13458,0.477
2,Lineups,-201568-201599-201933-201976-203085-,D. Gallinari - D. Jordan - B. Griffin - P. Bev...,1610612746,LAC,8,4,4,0.500,52.7,...,6331,14534,3156,3432,15661,11839,3366,4276,5097,0.554
3,Lineups,-201577-203200-1626170-1627835-1628374-,R. Lopez - J. Holiday - J. Grant - P. Zipser -...,1610612741,CHI,12,1,11,0.083,52.1,...,2864,12960,3855,4369,14548,12408,6359,4587,13927,0.428
4,Lineups,-202083-203084-1626257-1627812-1628467-,W. Matthews - H. Barnes - S. Mejri - Y. Ferrel...,1610612742,DAL,16,5,11,0.313,52.0,...,4964,11460,4365,3510,14748,12845,6195,4671,4626,0.538


In [40]:
for index, row in dfLineup.iterrows():
    playerids = row['GROUP_ID'].split("-")[1:-1]
    teamscore = 0
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
        playerscore = playerstats.iloc[0]['AST']
        teamscore = teamscore + playerscore
    dfLineup.at[index,'SUM_AST'] = teamscore     
    
dfLineup['AST_NET'] = dfLineup['AST'] - dfLineup['SUM_AST']

In [41]:
for index, row in dfLineup.iterrows():
    playerids = row['GROUP_ID'].split("-")[1:-1]
    teamscore = 0
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
        playerscore = playerstats.iloc[0]['FG_PCT']
        teamscore = teamscore + playerscore
    dfLineup.at[index,'SUM_FG_PCT'] = teamscore     
    
dfLineup['FG_PCT_NET'] = dfLineup['FG_PCT'] - dfLineup['SUM_FG_PCT']

In [42]:
for index, row in dfLineup.iterrows():
    playerids = row['GROUP_ID'].split("-")[1:-1]
    teamscore = 0
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
        playerscore = playerstats.iloc[0]['FG3_PCT']
        teamscore = teamscore + playerscore
    dfLineup.at[index,'SUM_FG3_PCT'] = teamscore     
    
dfLineup['FG3_PCT_NET'] = dfLineup['FG3_PCT'] - dfLineup['SUM_FG3_PCT']

In [43]:
for index, row in dfLineup.iterrows():
    playerids = row['GROUP_ID'].split("-")[1:-1]
    teamscore = 0
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
        playerscore = playerstats.iloc[0]['REB']
        teamscore = teamscore + playerscore
    dfLineup.at[index,'SUM_REB'] = teamscore     
    
dfLineup['REB_NET'] = dfLineup['REB'] - dfLineup['SUM_REB']

In [44]:
for index, row in dfLineup.iterrows():
    playerids = row['GROUP_ID'].split("-")[1:-1]
    teamscore = 0
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
        playerscore = playerstats.iloc[0]['TOV']
        teamscore = teamscore + playerscore
    dfLineup.at[index,'SUM_TOV'] = teamscore     
    
dfLineup['TOV_NET'] = dfLineup['TOV'] - dfLineup['SUM_TOV']

In [45]:
for index, row in dfLineup.iterrows():
    playerids = row['GROUP_ID'].split("-")[1:-1]
    teamscore = 0
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
        playerscore = playerstats.iloc[0]['STL']
        teamscore = teamscore + playerscore
    dfLineup.at[index,'SUM_STL'] = teamscore     
    
dfLineup['STL_NET'] = dfLineup['STL'] - dfLineup['SUM_STL']

In [46]:
for index, row in dfLineup.iterrows():
    playerids = row['GROUP_ID'].split("-")[1:-1]
    teamscore = 0
    for player in playerids:
        playerstats = dfPlayer.loc[dfPlayer['PLAYER_ID'] == int(player)]
        playerscore = playerstats.iloc[0]['PTS']
        teamscore = teamscore + playerscore
    dfLineup.at[index,'SUM_PTS'] = teamscore     
    
dfLineup['PTS_NET'] = dfLineup['PTS'] - dfLineup['SUM_PTS']

In [49]:
xvar = dfLineup[['AST_NET','FG_PCT_NET','FG3_PCT_NET','REB_NET','TOV_NET','STL_NET','PTS_NET','PIE']].copy()

In [50]:
model = sm.OLS(dfLineup['PLUS_MINUS'],xvar).fit()
#predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PLUS_MINUS   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.865
Method:                 Least Squares   F-statistic:                     201.7
Date:                Tue, 19 Feb 2019   Prob (F-statistic):          1.82e-102
Time:                        19:04:35   Log-Likelihood:                -745.88
No. Observations:                 250   AIC:                             1508.
Df Residuals:                     242   BIC:                             1536.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
AST_NET        -0.0482      0.087     -0.553      0.581      -0.220       0.124
FG_PCT_NET     34.1697      1.459     23.413      0.000      31.295      37.045
FG3_PCT_NET     7.5566      1.353      5.585      0.000       4.891      10.222
REB_NET         0.1301      0.070      1.871      0.063      -0.007       0.267
TOV_NET        -0.3526      0.127     -2.771      0.006      -0.603      -0.102
STL_NET         0.0822      0.179      0.460      0.646      -0.270       0.434
PTS_NET        -0.0069      0.036     -0.191      0.849      -0.078       0.064
PIE           147.2525      4.488     32.809      0.000     138.412     156.093
==============================================================================
Omnibus:                        6.637   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.036   Jarque-Bera (JB):                6.899
Skew:                          -0.293   Prob(JB):                       0.0318
Kurtosis:                       3.565   Cond. No.                         194.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',




SyntaxError: invalid syntax (<ipython-input-51-717068f4a3bf>, line 1)

In [53]:
from nba_api.stats.endpoints import leaguedashlineups

one = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Advanced', season = '2017-18', per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
print(list(one.columns))

two = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Base', season = '2017-18',per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
print(list(two.columns))

three = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Misc',season = '2017-18',per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
print(list(three.columns))

time.sleep(3)

four = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Four Factors',season = '2017-18',per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
print(list(four.columns))
five = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Scoring',season = '2017-18',per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
print(list(five.columns))
six = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Opponent',season = '2017-18',per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
print(list(six.columns))

#seven = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Usage',season = '2017-18',per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
#print(list(seven.columns))
#eight = leaguedashlineups.LeagueDashLineups(measure_type_detailed_defense = 'Defense',season = '2017-18',per_mode_detailed = 'Per100Possessions').get_data_frames()[0]
#print(list(eight.columns))

['GROUP_SET', 'GROUP_ID', 'GROUP_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'PIE', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK', 'NET_RATING_RANK', 'AST_PCT_RANK', 'AST_TO_RANK', 'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK', 'REB_PCT_RANK', 'TM_TOV_PCT_RANK', 'EFG_PCT_RANK', 'TS_PCT_RANK', 'PACE_RANK', 'PIE_RANK']
['GROUP_SET', 'GROUP_ID', 'GROUP_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RA